Попробуем решить следующую задачу: нам необходимо выгрузить сайт NPlus1.ru и положить статьи в базу данных. Отдельно должны лежать идентификаторы загруженных текстов, словари начальных форм и токенов, разбиение текста на предложения. Для решения задачи будем использовать базу данных MongoDB.

Импортируем все необходисые библиотеки. Обратите особое внимание на библиотеку для работы с Mongo - <a href="http://api.mongodb.com/python/current/api/index.html">pymongo</a> (еще одно описание <a href="https://pymongo.readthedocs.io/en/stable/">здесь</a>).

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import datetime
from tqdm import tqdm
import pymorphy2

import pymongo
from bson import ObjectId

delcom=re.compile("<!--.+-->", re.S)


Чтобы не ставить себе базу данных, можно использовать бесплатное облако <a href ="https://cloud.mongodb.com/">Mongo.Atlas</a>. Для этого необходимо зарегистрироваться, завести пользователя, создать проект, дать пользователю права на проект. Для пользователя получить строку для соединения с базой. 

Но в данном случае мы будем пользоваться MongoDB, запущенное на локальном компьютере при помощи Docker.

Для работы с БД используем класс MongoClient, в который передаем строку соединения.

In [2]:
# Version after 3.6
#client = pymongo.MongoClient('localhost', 27017)
client = pymongo.MongoClient()

Аналогом реляционных записей являются документы. Аналогом таблиц - коллекции и подколлекции. Сам документ может включать в себя другие документы. Документ записывается в формате JSON, но для Питона можно считать, что это словари.<br>
При помощи клиента обращаемся к базе `concordance2`. Если эта база не существовала, она создастся при первой записи данных. Аналогично всё происходит с коллекциями.<br>

В коллекции `nplus1texts` будем хранить адерса и идентификаторы текстов, в коллекции `dictionary` - словарь токенов, в `lemmas` - словарь лемм, наконец в `sentences` - разделение предложений на слова.

In [3]:
# Обращение к базе при помощи оператора квадратные скобки.
db = client['concordance']
# Обращение к базе при помощи оператора точка.
#db = client.concordance

Посмотрим что содержит в себе база.

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'concordance')

In [5]:
# Обращение к коллекции внутри базы при помощи оператора точка (но можно и при помощи квадратных скобок).
text_collection=db.nplus1texts
dictionary=db.dictionary
lemmas=db.lemmas
dbsents=db.sentences

In [6]:
# Добавляем индекс в коллекцию lemmas по полям iniForm и POS, оба по возрастанию.
# Заодно запрещаем добавлять повторяющиеся сочетания.
lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)
dictionary.create_index([('token', pymongo.ASCENDING), ('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

'token_1_iniForm_1_POS_1'

In [7]:
# Для пробы чуть позже удалим индекс и посмотрим на изменение скорости работы.
lemmas.drop_index('iniForm_1_POS_1')

Напишем функцию для выгрузки одной статьи с сайта (это мы уже умеем).

In [7]:
class NPlus1Article:
    def __init__(self):
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        self.head=""
        self.text=""

def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    #print(r.text)
    art=NPlus1Article()
    tables=re.split("</div>", re.split('="tables"', r.text)[1])[0]
    t1=re.split("</time>", re.split("<time", tables)[1])[0]
    art.time=re.split("</span>", re.split("<span>", t1)[1])[0]
    art.date=re.split("</span>", re.split("<span>", t1)[2])[0]
    art.rubr=re.split(">", re.split("</a>", re.split("<a href", tables)[1])[0])[1]
    art.diff=re.split("</span>", re.split('"difficult-value">', tables)[1])[0]
    art.head=re.split("</h1>", re.split('<h1>', r.text)[1])[0]
    art.author=re.split('" />', re.split('<meta name="author" content="', r.text)[1])[0]
    art.text=re.split("</div>", re.split("</figure>", re.split('</article>',re.split('<article', r.text)[1])[0])[1])[1]    

    beaux_text=BeautifulSoup(art.text, "html5lib")
    art.text=delcom.sub("", beaux_text.get_text() )

    # print(art.n_time, art.n_date, art.n_rubr, art.n_diff)
    # print(art.n_head)
    # print(art.n_author)
    # print(art.n_text)
    #return [n_time, n_date, n_rubr, n_diff, n_author, n_head, n_text]
    return art

Кстати, requests умеет скачивать что угодно. Например, pdf-файл.

In [19]:
r = requests.get("https://riptutorial.com/Download/json-ru.pdf")

In [22]:
r.text[:100]


'%PDF-1.7\n%����\n311 0 obj\n<</Filter/FlateDecode/Length 107>>stream\nx�+�2�4U0\x00B\x0b\x13#0��˥�k����\x15�U��\x14¥�f�'

Для того, чтобы найти документ, необходимо использовать функцию `collection.find()`. Без параметров эта функция выдаст все документы.

In [8]:
for t in text_collection.find():
    print(t)

{'_id': ObjectId('5f818f46cc825874ec349c33'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.Старт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсохода будет\xa0поиск соединени

В качестве параметров функции передается документ (в случае Python - словарь) с полями и их значениями. 

Если значение должно отвечать не равенству, а другому условию, передается составное значение в виде словаря, ключем у которого будет обозначение оператора, а значением - значение с которым будет проводиться сравнение. Таких пар можно передать несколько.

In [11]:
# (freq _какое_ 2) _и_ (token _какое_ 'может')
#for l in dictionary.find({"freq": 2, 'token': 'может'}):
# token _какое_ 'может'
for l in dictionary.find({'token': 'может', "freq": 9}):
    print(l)


{'_id': ObjectId('5f818f49cc825874ec34a12b'), 'token': 'может', 'iniForm': 'мочь', 'POS': 'VERB', 'freq': 9, 'docs': [ObjectId('5f818f48cc825874ec349e82'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5f818f54cc825874ec34ad4c')]}


In [12]:
# freq _какое_ меньше _чего_ 5
for l in lemmas.find({"freq":{"$gt": 5}})[:5]:
    print(l)

print("----")
# freq _какое_ меньше _чего_ 5
for l in dictionary.find({"freq":{"$gt": 5}})[:5]:
    print(l)
    
print("----")
# freq _какое_ ((больше _чего_ 5) _и_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5, "$lt": 9}})[:5]:
    print(l)

{'_id': ObjectId('5f818f47cc825874ec349c3d'), 'iniForm': 'который', 'POS': 'ADJF', 'freq': 37, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f48cc825874ec349e82'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5f818f4fcc825874ec34a890'), ObjectId('5f818f51cc825874ec34aa67'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f818f54cc825874ec34ad4c'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f818f47cc825874ec349c40'), 'iniForm': 'должный', 'POS': 'ADJS', 'freq': 7, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f4fcc825874ec34a890'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f818f47cc825874ec349c46'), 'iniForm': 'к', 'POS': 'PREP', 'freq': 17, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f48cc825874ec349e82'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5f818f51cc825874ec34aa67'), ObjectId(

Те же запросы можно представить в виде вот таких деревьев.

```
{"freq":{"$ge": 5}}  
  
freq  
  |  
  ge  
  |  
  5  
  
   ge  
  /  \  
freq  5  
  
{"freq":{"$gt": 5, "$lt": 9}}  
  
freq  
   |  
   &  
  /  \  
 gt  lt  
  |   |  
  5   9  
    
{"freq":{"$gt":5,"$lt":9}}  
```

In [13]:
res1 = lemmas.find({"freq":{"$gt": 5}})
res1[0]['iniForm']

'который'

In [14]:
# Можно записать и так.
# (freq _какое_ (больше _чего_ 5)() _и_ (freq _какое_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5}, "freq": {"$lt": 9}}):
    print(l)

{'_id': ObjectId('5f818f46cc825874ec349c35'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f818f46cc825874ec349c33')]}
{'_id': ObjectId('5f818f47cc825874ec349c38'), 'token': 'Розалинд', 'iniForm': 'розалинда', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('5f818f46cc825874ec349c33')]}
{'_id': ObjectId('5f818f47cc825874ec349c3b'), 'token': 'Франклин', 'iniForm': 'франклин', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('5f818f46cc825874ec349c33')]}
{'_id': ObjectId('5f818f47cc825874ec349c3e'), 'token': 'который', 'iniForm': 'который', 'POS': 'ADJF', 'freq': 5, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f818f47cc825874ec349c41'), 'token': 'должен', 'iniForm': 'должный', 'POS': 'ADJS', 'freq': 3, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4fcc825874ec34a890')]}
{'_id': ObjectId('5f818f47cc825874ec349c44'), 'token': 'отправиться', 

{'_id': ObjectId('5f818f4ecc825874ec34a709'), 'token': 'хуже', 'iniForm': 'худой', 'POS': 'COMP', 'freq': 1, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5')]}
{'_id': ObjectId('5f818f4ecc825874ec34a70d'), 'token': 'третьем', 'iniForm': 'третье', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5')]}
{'_id': ObjectId('5f818f4ecc825874ec34a711'), 'token': 'участвовали', 'iniForm': 'участвовать', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5')]}
{'_id': ObjectId('5f818f4ecc825874ec34a718'), 'token': 'проходил', 'iniForm': 'проходить', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5')]}
{'_id': ObjectId('5f818f4ecc825874ec34a71c'), 'token': 'дневное', 'iniForm': 'дневный', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5')]}
{'_id': ObjectId('5f818f4ecc825874ec34a721'), 'token': 'продолжительность', 'iniForm': 'продолжительность', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5')]}
{'_

{'_id': ObjectId('5f8435193c5cb274c4b37f68'), 'token': 'облегчает', 'iniForm': 'облегчать', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f8435193c5cb274c4b37f6b'), 'token': 'адсорбцию', 'iniForm': 'адсорбция', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f8435193c5cb274c4b37f6e'), 'token': 'кислород-содержащих', 'iniForm': 'кислород-содержимый', 'POS': 'PRTF', 'freq': 1, 'docs': [ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f8435193c5cb274c4b37f70'), 'token': 'частиц', 'iniForm': 'частица', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f8435193c5cb274c4b37f75'), 'token': 'низкое', 'iniForm': 'низкий', 'POS': 'ADJF', 'freq': 2, 'docs': [ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f8435193c5cb274c4b37f79'), 'token': 'анодного', 'iniForm': 'анодный', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('5f8435183c5cb274c4b37f3d')]

Попробуем найти все документы со сложностью равной 6.8 (статья про F-35).

In [11]:
for l in text_collection.find({'art_info': {'difficulty': '6.8'}}):
    print(l)

Ничего не найдено, так как такая запись предполагает полное совпадение, то есть поиск поддокумента с единственным полем и этим значением. Если полей будет несколько, роль будет играть даже порядок следования полей в базе!

То, что мы хотели сделать,нужно сделать вот так.

In [15]:
for l in text_collection.find({'art_info.difficulty': '6.8'}):
    print(l)

{'_id': ObjectId('5f818f56cc825874ec34af2c'), 'text_url': 'https://nplus1.ru/news/2018/03/20/abel-prize-2018', 'text_name': 'Абелевская премия присуждена за открытие связи между теорией чисел и теорией представлений', 'art_text': 'Норвежская академия наук объявила лауреата Абелевской премии 2018 года. Им стал канадский математик\xa0Роберт Ленглендс. Премия присуждена «за дальновидную программу, соединяющую теорию представлений и теорию чисел».\xa0Абелевскую премию часто называют «Нобелевской премией по математике». В отличие от, например, Филдсовской медали она вручается каждый год. Размер премии — 6 миллионов крон (около 45 миллионов рублей). Церемония награждения пройдет 22 мая 2018 года в Университете Аула, Осло. Вручать награду будет лично король Норвегии Харальд V.\n    \n        \n                        \n                \n                    \n                        \n                    ', 'art_info': {'art_date': '20 Март 2018', 'art_time': '14:15', 'difficulty': '6.8', 'aut

Можно выбирать не все поля, а только те, которые нам необходимы.

In [13]:
for t in text_collection.find(projection=['art_info','text_name']):
    print(t)

{'_id': ObjectId('5f818f46cc825874ec349c33'), 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания', 'art_info': {'art_date': '20 Янв. 2020', 'art_time': '14:18', 'difficulty': '2.3', 'author': 'Александр Войтюк'}}
{'_id': ObjectId('5f818f48cc825874ec349e82'), 'text_name': 'Физики заперли свет в нанорезонаторе на рекордно долгое время', 'art_info': {'art_date': '20 Янв. 2020', 'art_time': '10:40', 'difficulty': '4.2', 'author': 'Олег Макаров'}}
{'_id': ObjectId('5f818f4acc825874ec34a172'), 'text_name': 'Пилотируемый полет Crew Dragon к МКС состоится в первой половине 2020 года', 'art_info': {'art_date': '20 Янв. 2020', 'art_time': '11:56', 'difficulty': '1.7', 'author': 'Григорий Копиев'}}
{'_id': ObjectId('5f818f4ccc825874ec34a3f5'), 'text_name': 'Сон помог вознаграждению улучшить зрительное обучение', 'art_info': {'art_date': '18 Янв. 2020', 'art_time': '15:28', 'difficulty': '3.9', 'author': 'Елизавета Ивтушок'}}
{'_id': ObjectId('5f818f4fcc825874ec34a890'),

Для поиска в массивах одного значения можно просто проводить поиск, БД сама будет искать объекты в массиве.

Если мы ищем несколько значений в массиве, необходимо использовать соответствующие предикаты: `$in` для поиска вхождения одного из элементов, `$all` - поиска всех элементов, и т.д.

Обратите внимание, для поиска мы используем не строку, а объект `ObjectId`, импортированный из библиотеки `bson`.

Помимо этого, во втором цикле в поле projection мы передаем не только какие поля надо выводить, но и какие поля выводить не надо. Так, поле \_id, хранящее идентификатор документа, по умолчанию должно выводиться. В примере ниже оно подавляется при помощи `"_id":False`.

In [50]:
for d in dictionary.find({"docs": ObjectId('5f818f46cc825874ec349c33')})[:5]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$in": [ObjectId('5f818f46cc825874ec349c33')]}})[:5]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$all": [ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f818f53cc825874ec34acc5')]}},
                         projection={"_id":False, "token":True, "docs":True}):
    print(d)    

{'_id': ObjectId('5f818f46cc825874ec349c35'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f818f46cc825874ec349c33')]}
{'_id': ObjectId('5f818f47cc825874ec349c38'), 'token': 'Розалинд', 'iniForm': 'розалинда', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('5f818f46cc825874ec349c33')]}
{'_id': ObjectId('5f818f47cc825874ec349c3b'), 'token': 'Франклин', 'iniForm': 'франклин', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('5f818f46cc825874ec349c33')]}
{'_id': ObjectId('5f818f47cc825874ec349c3e'), 'token': 'который', 'iniForm': 'который', 'POS': 'ADJF', 'freq': 3, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f52cc825874ec34ab00')]}
{'_id': ObjectId('5f818f47cc825874ec349c41'), 'token': 'должен', 'iniForm': 'должный', 'POS': 'ADJS', 'freq': 3, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4fcc825874ec34a890')]}
-----
{'_id': ObjectId('5f818f46cc825874ec349c35'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN

In [20]:
# А это мы просто смотрим сколько текстов добавили в базу.
# В качестве параметра передается условие, считаются только удовлетворяющие ему документы.
# В данном случае нам нужны все.
print(text_collection.count_documents({}))

# Ищем количество слов с частотой встречаемости от 6 до 9.
print(lemmas.count_documents({"freq":{"$gt": 5, "$lt": 10}}))

# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим не выводить идентификатор и отсортировать всё по частоте.
for l in lemmas.find({"freq":{"$gt": 5, "$lt": 10}}, {"_id":False}).sort("freq")[:5]:
    print(l)
print("----")
# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим вывести токен и частоту, но не выводить идентификатор и отсортировать всё по частоте.
for l in dictionary.find({"freq":{"$gt": 5}}, {"token": True, "freq":True, "_id":False}).sort("freq")[:5]:
    print(l)


11
45
{'iniForm': 'показывать', 'POS': 'VERB', 'freq': 6, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f53cc825874ec34acc5'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'iniForm': 'исследование', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4fcc825874ec34a890'), ObjectId('5f818f51cc825874ec34aa67'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'iniForm': 'пока', 'POS': 'ADVB', 'freq': 6, 'docs': [ObjectId('5f818f48cc825874ec349e82'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'iniForm': 'резонатор', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('5f818f48cc825874ec349e82')]}
{'iniForm': 'качество', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f8435183c5cb274c4b37f3d')]}
----
{'token': 'света', 'freq': 6}
{'token': 'два', 'freq': 6}
{'token': 'руководст

Теперь напишем функцию для того, чтобы класть статью в БД. используем для этого функции:
- insert_one (insert, insert_many) - добавляет запись (несколько записей) в выбранную коллекцию.
- find_one_and_update - найти и обновить (аналог update).

В качестве параметра во все функции добавления передается документ (словарь), ключи которого содержат названия полей, а значения - значения этих полей. Значение поля само может являться документом. При поиске часть полей может принимать особые значения, например, `$ne` - не равно, `$size` - смотрим на размер массива, `$inc` - увеличить значение поля на заданную величину и т.д. Более подробно с соответствующими ключами можно ознакомиться <a href="https://metanit.com/nosql/mongodb/" >здесь</a>.

In [22]:
for d in dictionary.find({"docs": {"$size": 3}})[:5]:
    print(d)


{'_id': ObjectId('5f818f47cc825874ec349c3e'), 'token': 'который', 'iniForm': 'который', 'POS': 'ADJF', 'freq': 5, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f818f47cc825874ec349c62'), 'token': 'испытания', 'iniForm': 'испытание', 'POS': 'NOUN', 'freq': 4, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f54cc825874ec34ad4c')]}
{'_id': ObjectId('5f818f47cc825874ec349c85'), 'token': 'сайте', 'iniForm': 'сайт', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f51cc825874ec34aa67'), ObjectId('5f818f52cc825874ec34ab00')]}
{'_id': ObjectId('5f818f47cc825874ec349d03'), 'token': 'работы', 'iniForm': 'работа', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f53cc825874ec34acc5')]}
{'_id': ObjectId('5f818f47cc825874ec349d44'), 'toke

In [23]:
# Создаем морфоанализатор.
morph=pymorphy2.MorphAnalyzer()

In [24]:
def putNPlus1ArticleInMongo(art_url):
    # Загружаем текст статьи и другие ее части.
    art = getArticleTextNPlus1(art_url) 

    # Добавляем запись с информацией о статье.
    # inserted_id позволяет сразу получить идентификатор записи, чтобы потом ссылаться на него там, где это необходимо.
    a_text = {"text_url": art_url, "text_name": art.head, "art_text": art.text, 
              "art_info": {"art_date": art.date, "art_time": art.time, 
                           "difficulty": art.diff, "author": art.author}}
    text_id = text_collection.insert_one(a_text).inserted_id
    print(text_id)

    # Выделяем предложения (просто по точке с пробелом или в конце строки!!!).
    sents = re.split("\.\s|\.$", art.text)
    sent_num = 1
    # Загружаем предложения в базу.
    for s in tqdm(sents):
        # Выделяем слова (просто как группы русских букв!!!).
        words = re.findall("([А-Яа-я]+(\-[А-Яа-я]+)?)", s)
        posit = 1
        # Загрузка слов из предложений.
        for w in words:
            wf = morph.parse(w[0])
            # Провели морфологический анализ и теперь добавляем документ с двумя полями: нач. форма и часть речи.
            # Сперва проверяем есть ли там уже такая запись.
            # Смотрим сколько таких слов нашлось. Если ни одного - надо добавлять.
            if lemmas.count_documents({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                lemma_id = lemmas.insert_one({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS, 
                                              "freq": 1, "docs":[text_id]}).inserted_id
            # А если такое слово уже было, то обновляем частоту встречаемости и в каком документе оно встретиось.
            else:
                inis = lemmas.find({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
                lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}, "$addToSet": {"docs": text_id}})
                # Вот таким образом можно добавлять номер документа столько раз, сколько в нем встретилось слово.
                #lemmas.find_one_and_update({"_id": lemma_id}, {"$push": {"docs": text_id}}) 

            # Повторяем операцию для токенов.
            if dictionary.count_documents({"token":w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                wf_id = dictionary.insert_one({"token":w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS, "freq": 1, "docs": [text_id]}).inserted_id
            else:
                wrdf = dictionary.find({"token": w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                wf_id=wrdf[0]["_id"]
                dictionary.find_one_and_update({"_id": wf_id}, {"$inc":  {"freq": 1}}) 
                dictionary.find_one_and_update({"_id": wf_id}, {"$addToSet": {"docs": text_id}}) 
  
            # Добавляем номер предложения, идентификатор словоформы из словаря, позицию слова, из какого оно текста.
            dbsents.insert_one({"sent_id": sent_num, "wordFormId":wf_id, "position": posit, "textId": text_id})
            posit += 1  
        sent_num += 1

In [25]:
# Добавляем статью  в базу.
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/subwavelength-resonators")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/crew-dragon-first-crewed-flight")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/visual-perception")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/dancing-recognition")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/27/juno-pictures")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/04/12/alignment-of-jets")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/29/Ultra-Deep-Survey")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/balance")
# putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/abel-prize-2018")
#putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/10/12/srb-electrode")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/11/11/physical-fitness")

  0%|          | 0/16 [00:00<?, ?it/s]

5fac07a39c86b17f5e69cac0


100%|██████████| 16/16 [00:03<00:00,  5.06it/s]


In [ ]:
# Вот так можно удалить всё в базе. Если написать условие, то не всё. 
# Экспериментировать не будем, да?
text_collection.delete_many({})
dictionary.delete_many({})
lemmas.delete_many({})
dbsents.delete_many({})
#for l in dbsents.find():
# print(l)


In [28]:
# Так можно посмотреть все записи, у которых в токене записан список ровно из пяти элементов.
#for l in dictionary.find({"docs":{"$size":5}})[:5]:
for l in dictionary.find({"docs":{"$size":4}})[:5]:
    print(l)
print("=====")
# Так можно посмотреть все записи, у которых есть поле docs.
for l in lemmas.find({"docs":{"$exists":True}})[:5]:
    print(l)

{'_id': ObjectId('5f818f47cc825874ec349c3e'), 'token': 'который', 'iniForm': 'который', 'POS': 'ADJF', 'freq': 6, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f8435183c5cb274c4b37f3d'), ObjectId('5fac07a39c86b17f5e69cac0')]}
{'_id': ObjectId('5f818f47cc825874ec349cad'), 'token': 'года', 'iniForm': 'год', 'POS': 'NOUN', 'freq': 10, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f48cc825874ec349e82'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f56cc825874ec34af2c')]}
{'_id': ObjectId('5f818f47cc825874ec349cfd'), 'token': 'качестве', 'iniForm': 'качество', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f8435183c5cb274c4b37f3d')]}
{'_id': ObjectId('5f818f47cc825874ec349d44'), 'token': 'о', 'iniForm': 'о', 'POS': 'PREP', 'freq': 8, 'docs': [ObjectId('5f818f46cc825874ec349c33'), ObjectId('5f818f52cc825874e

При помощи ключевого слова `$regex' можно искать в базе строки, отвечающие регулярным выражениям.

In [29]:
# iniForm _какое_ описывается_регуляркой _какой_ ^и.+
for d in lemmas.find({'iniForm':{'$regex':'^и.+'}})[:5]:
    print(d)
print("====")    
for d in dictionary.find({'token':{'$regex':'^[А-Я].+'}})[:5]:
    print(d)

{'_id': ObjectId('5f818f4fcc825874ec34a88d'), 'iniForm': 'ивтушок', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5fac07a39c86b17f5e69cac0')]}
{'_id': ObjectId('5f818f4ccc825874ec34a44f'), 'iniForm': 'играть', 'POS': 'VERB', 'freq': 2, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5')]}
{'_id': ObjectId('5f818f4bcc825874ec34a386'), 'iniForm': 'идея', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f818f4acc825874ec34a172')]}
{'_id': ObjectId('5f818f49cc825874ec349fd5'), 'iniForm': 'из', 'POS': 'PREP', 'freq': 25, 'docs': [ObjectId('5f818f48cc825874ec349e82'), ObjectId('5f818f4acc825874ec34a172'), ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5f818f4fcc825874ec34a890'), ObjectId('5f818f51cc825874ec34aa67'), ObjectId('5f818f52cc825874ec34ab00'), ObjectId('5f818f54cc825874ec34ad4c'), ObjectId('5f8435183c5cb274c4b37f3d'), ObjectId('5fac07a39c86b17f5e69cac0')]}
{'_id': ObjectId('5f818f54cc825874ec34add6'), 'iniForm': 'из-за', 'POS': 'PREP', 'freq': 2, 'docs'

Ключевое слово `$or` и другие операторы позволяют применять логические свзки помимо логического И.

In [30]:
# _или_по_списку_условий_ [token _какой_ описывается_регуляркой _какой_ ^И.+, 
#                          token _какой_ описывается_регуляркой _какой_ .+ъ.+]
for d in dictionary.find({'$and':[{'token':{'$regex':'^о.+'}}, {'token':{'$regex':'.+ъ.+'}}]})[:5]:
    print(d)

{'_id': ObjectId('5f818f50cc825874ec34a9cf'), 'token': 'объекты', 'iniForm': 'объект', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('5f818f4fcc825874ec34a890')]}
{'_id': ObjectId('5fac07a59c86b17f5e69cc26'), 'token': 'объем', 'iniForm': 'объесть', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('5fac07a39c86b17f5e69cac0')]}
{'_id': ObjectId('5f818f56cc825874ec34af36'), 'token': 'объявила', 'iniForm': 'объявить', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('5f818f56cc825874ec34af2c')]}
{'_id': ObjectId('5f818f4bcc825874ec34a310'), 'token': 'объявило', 'iniForm': 'объявить', 'POS': 'VERB', 'freq': 1, 'docs': [ObjectId('5f818f4acc825874ec34a172')]}


Обратите внимание, что при обновлении заменяетя весь документ целиком. Чтобы избежать этого следует использовать ключевое слово `$set`, которое показывает, что обновляется значение только одного поля.

In [34]:
# Просто посмотрим на документы
for d in dictionary.find({'token':{'$regex':'^И.+'}}):
    print(d)

{'_id': ObjectId('5f818f49cc825874ec34a04a'), 'token': 'ИТМО', 'iniForm': 'итмый', 'POS': 'ADJS', 'freq': 1, 'docs': [ObjectId('5f818f48cc825874ec349e82')], 'found': 'true'}
{'_id': ObjectId('5f818f4fcc825874ec34a88e'), 'token': 'Ивтушок', 'iniForm': 'ивтушок', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('5f818f4ccc825874ec34a3f5'), ObjectId('5fac07a39c86b17f5e69cac0')], 'found': 'true'}
{'_id': ObjectId('5f818f49cc825874ec34a087'), 'token': 'Из', 'iniForm': 'из', 'POS': 'PREP', 'freq': 1, 'docs': [ObjectId('5f818f48cc825874ec349e82')], 'found': 'true'}
{'_id': ObjectId('5f8435193c5cb274c4b3803a'), 'token': 'Известно', 'iniForm': 'известно', 'POS': 'PRED', 'freq': 1, 'docs': [ObjectId('5f8435183c5cb274c4b37f3d')], 'found': 'true'}
{'_id': ObjectId('5f818f4bcc825874ec34a33a'), 'token': 'Изначально', 'iniForm': 'изначально', 'POS': 'ADVB', 'freq': 1, 'docs': [ObjectId('5f818f4acc825874ec34a172')], 'found': 'true'}
{'_id': ObjectId('5f818f4acc825874ec34a199'), 'token': 'Илон', 'iniForm': 

In [31]:
# А теперь найдем первое попавшееся слово, начинающееся с заглавное И и добавим ему новое поле found
dictionary.find_one_and_update({'token':{'$regex':'^И.+'}}, {'$set': {'found':'true'}})

{'_id': ObjectId('5f818f49cc825874ec34a04a'),
 'token': 'ИТМО',
 'iniForm': 'итмый',
 'POS': 'ADJS',
 'freq': 1,
 'docs': [ObjectId('5f818f48cc825874ec349e82')],
 'found': 'true'}

In [33]:
dictionary.find_one_and_update({'token':{'$regex':'^И.+'}, 'found':{"$exists":False}}, {'$set': {'found':'true'}})

In [36]:
# Попробуем найти все документы с полем found.
for d in dictionary.find({'token':{'$regex':'^И.+'}, 'found':{"$exists":False}}):
    print(d)

#### Важное замечание

Вся структура нашей базы данных отдает реляционностью. Никто не мешал нам сделать документ, вложенный в словарь начальных форм и хранить в нем все формы данного слова. Тогда если что, не надо будет делать запросы, в которых надо делать пересечение двух коллекций.

## Работа с географическими данными

MongoDB позволяет работать с географическими данными. Основой является [GEO JSON](https://docs.mongodb.com/manual/reference/geojson/), позволяющий обрабатывать разные виды геолокаций.


In [22]:
text_collection.find_one_and_update(
    {'_id': ObjectId('5f8435183c5cb274c4b37f3d')},
    {"$set":{'coord':{ 'type': "Point", 'coordinates': [ 20, 15 ] }}})


{'_id': ObjectId('5f8435183c5cb274c4b37f3d'),
 'text_url': 'https://nplus1.ru/news/2020/10/12/srb-electrode',
 'text_name': 'Бактерии помогли получить катализатор для электролиза воды',
 'art_text': 'Китайские\nхимики получили электроды для электролиза\nводы с\xa0помощью сульфатредуцирующих\nбактерий. Бактерии покрывают поверхность\nэлектрода сульфидом железа, который\nзатем облегчает адсорбцию кислород-содержащих\nчастиц. Полученные электроды показывают\nнизкое значение анодного перенапряжение\nв\xa0220\xa0милливольт, а\xa0сам процесс их\xa0получения\nочень прост — его можно будет легко\nадаптировать для промышленности.\nРезультаты исследования опубликованы\nв\xa0журнале Nature\nCommunications.Один\nиз\xa0наиболее\nэкологичных способов получения\nводородного топлива\xa0— электролиз воды.\nЧтобы\nтратить на\xa0этот процесс меньше энергии,\nученые покрывают поверхность электродов\nразными катализаторами. Для анода (на\nнем при электролизе выделяется кислород)\nочень эффективными оказали

In [23]:
for a in text_collection.find():
    print(a)

{'_id': ObjectId('5f818f46cc825874ec349c33'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': 'Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.Старт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсохода будет\xa0поиск соединени

In [10]:
for d in text_collection.find({'coord':{'$geoIntersects':{"$geometry": 
{"type": "Polygon",
 "coordinates": [[[15,10], [15,20],[25,20], [25,10], [15,10]]]
}}}}):
    print(d)

{'_id': ObjectId('5f8435183c5cb274c4b37f3d'), 'text_url': 'https://nplus1.ru/news/2020/10/12/srb-electrode', 'text_name': 'Бактерии помогли получить катализатор для электролиза воды', 'art_text': 'Китайские\nхимики получили электроды для электролиза\nводы с\xa0помощью сульфатредуцирующих\nбактерий. Бактерии покрывают поверхность\nэлектрода сульфидом железа, который\nзатем облегчает адсорбцию кислород-содержащих\nчастиц. Полученные электроды показывают\nнизкое значение анодного перенапряжение\nв\xa0220\xa0милливольт, а\xa0сам процесс их\xa0получения\nочень прост — его можно будет легко\nадаптировать для промышленности.\nРезультаты исследования опубликованы\nв\xa0журнале Nature\nCommunications.Один\nиз\xa0наиболее\nэкологичных способов получения\nводородного топлива\xa0— электролиз воды.\nЧтобы\nтратить на\xa0этот процесс меньше энергии,\nученые покрывают поверхность электродов\nразными катализаторами. Для анода (на\nнем при электролизе выделяется кислород)\nочень эффективными оказались\

#### Важное замечание

!!! Если не построить этот индекс, то искаться не будет!!!

In [37]:
#lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

text_collection.create_index([("coord", pymongo.GEOSPHERE)], unique=False)

'coord_2dsphere'

In [38]:
req=text_collection.find(
{
   "coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ 19.999 , 15.001 ]
       },
       "$maxDistance": 1000,
       "$minDistance": 0
     }
   }
}
)
#req
for d in req:    
    print(d)

{'_id': ObjectId('5f8435183c5cb274c4b37f3d'), 'text_url': 'https://nplus1.ru/news/2020/10/12/srb-electrode', 'text_name': 'Бактерии помогли получить катализатор для электролиза воды', 'art_text': 'Китайские\nхимики получили электроды для электролиза\nводы с\xa0помощью сульфатредуцирующих\nбактерий. Бактерии покрывают поверхность\nэлектрода сульфидом железа, который\nзатем облегчает адсорбцию кислород-содержащих\nчастиц. Полученные электроды показывают\nнизкое значение анодного перенапряжение\nв\xa0220\xa0милливольт, а\xa0сам процесс их\xa0получения\nочень прост — его можно будет легко\nадаптировать для промышленности.\nРезультаты исследования опубликованы\nв\xa0журнале Nature\nCommunications.Один\nиз\xa0наиболее\nэкологичных способов получения\nводородного топлива\xa0— электролиз воды.\nЧтобы\nтратить на\xa0этот процесс меньше энергии,\nученые покрывают поверхность электродов\nразными катализаторами. Для анода (на\nнем при электролизе выделяется кислород)\nочень эффективными оказались\

In [28]:
for d in text_collection.find({'coord':{'$geoWithin':{"$box": 
[[15,10], [25,20]]
}}}):
    print(d)

{'_id': ObjectId('5f8435183c5cb274c4b37f3d'), 'text_url': 'https://nplus1.ru/news/2020/10/12/srb-electrode', 'text_name': 'Бактерии помогли получить катализатор для электролиза воды', 'art_text': 'Китайские\nхимики получили электроды для электролиза\nводы с\xa0помощью сульфатредуцирующих\nбактерий. Бактерии покрывают поверхность\nэлектрода сульфидом железа, который\nзатем облегчает адсорбцию кислород-содержащих\nчастиц. Полученные электроды показывают\nнизкое значение анодного перенапряжение\nв\xa0220\xa0милливольт, а\xa0сам процесс их\xa0получения\nочень прост — его можно будет легко\nадаптировать для промышленности.\nРезультаты исследования опубликованы\nв\xa0журнале Nature\nCommunications.Один\nиз\xa0наиболее\nэкологичных способов получения\nводородного топлива\xa0— электролиз воды.\nЧтобы\nтратить на\xa0этот процесс меньше энергии,\nученые покрывают поверхность электродов\nразными катализаторами. Для анода (на\nнем при электролизе выделяется кислород)\nочень эффективными оказались\

In [39]:
#text_collection.create_index([('text_name', pymongo.ASCENDING)], unique=False)
text_collection.create_index([('art_text', pymongo.TEXT)], unique=False)


'art_text_text'

In [43]:
#for d in text_collection.find({'text_name': 'Бактерии помогли получить катализатор для электролиза воды'}):
# for d in text_collection.find({
#   "$text":{"$search": "воды"}}):
        
#for d in text_collection.find({'art_text':{'$regex':'воды'}}):
for d in text_collection.find({ "$text": { "$search": "Американские ученые" } } ):
#for d in text_collection.find({ "$text": { "$search": "Американ" } } ):
    print(d)

{'_id': ObjectId('5f8435183c5cb274c4b37f3d'), 'text_url': 'https://nplus1.ru/news/2020/10/12/srb-electrode', 'text_name': 'Бактерии помогли получить катализатор для электролиза воды', 'art_text': 'Китайские\nхимики получили электроды для электролиза\nводы с\xa0помощью сульфатредуцирующих\nбактерий. Бактерии покрывают поверхность\nэлектрода сульфидом железа, который\nзатем облегчает адсорбцию кислород-содержащих\nчастиц. Полученные электроды показывают\nнизкое значение анодного перенапряжение\nв\xa0220\xa0милливольт, а\xa0сам процесс их\xa0получения\nочень прост — его можно будет легко\nадаптировать для промышленности.\nРезультаты исследования опубликованы\nв\xa0журнале Nature\nCommunications.Один\nиз\xa0наиболее\nэкологичных способов получения\nводородного топлива\xa0— электролиз воды.\nЧтобы\nтратить на\xa0этот процесс меньше энергии,\nученые покрывают поверхность электродов\nразными катализаторами. Для анода (на\nнем при электролизе выделяется кислород)\nочень эффективными оказались\